In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
import os

In [23]:
df = pd.read_feather('imgl.df').set_index('index').query('ds_type == "train"')
func = lambda s: s[s.find("images/")+7:]
df["local_path"] = np.array([[f"../seq_data/{s}/{func(i)}"for i in c] for c,s in zip(df.Paths,df.ds_type)]).tolist()

In [24]:
ns = df.groupby('seq_key')['loss'].count()
losses = df.groupby('seq_key')['loss'].mean().loc[ns > 3]

losses

seq_key
20GnnQaw7R2pR0a_ecPDnA     4.756585
2gqajn4lerxvknrsnx1fbm     3.986484
2taddw3rpf3tr95ze9ht1u    10.385349
6oz1jib283lzg70asqy2wx     3.367256
8ighk3bcrezamulw4vd3vd     3.057056
99tbbk7jfc8k8b17mono0y     5.404414
9Gx4zJlSqjhghpFlVcGU3Q     1.716154
GMWGQtu0jlAUXsRCE1Asag     4.333782
H7hbf3RBlB_tOOGqBfHDaA    10.313719
MHfiPqEc6AmLDW2xLCsixw     6.299242
UMZX7LdDAdkK6Qul-nA-1g     4.711794
UiGpDEQrzX_Q5PGB1aNiDg     7.062361
VK8smP0CTeOZmHlU2fWCeA     4.447124
daOCuCCLxkeZOqPpRX8PMA     8.764616
eFJyH89rnxO-lRYfJ7Q_QQ     7.842801
f49i4w199yp01usz30lx45     4.415981
gs2ymgvpz7g7qd4fn4mwpn     5.375650
ufNPIv1mpUKOii9oDVdGOw     9.078372
wRe_Ed0B1O5lIuk2DU8wUQ     2.898792
Name: loss, dtype: float32

In [39]:
# commands = []
# for seq in losses.index:

#     a = df.query(f'seq_key == "{seq}"')
#     name = f"tmp/seqs/{seq}Ä{a.view_direction.iloc[0]}Ä{losses.loc[seq]:.2f}"
#     os.makedirs(name,exist_ok=True)
#     imgs = "../seq_data/train/" + np.unique(np.concatenate(a['keys'].values))  + ".jpg"

#     paths = "cp " + imgs +  " " + name
#     commands.extend(paths.tolist())

# text = "\n".join(commands)

In [40]:
# with open('copys.sh',"w") as f:
#     f.write(text)

In [52]:
for seq in ns.loc[ns < 4].index:

    a = df.query(f'seq_key == "{seq}"')
    for index, row in a.iterrows():
        plot_row(row,f"tmp/saves/{index}.jpg")

In [50]:
def plot_row(row,save = None):
    fig, ax = plt.subplots(2,2,figsize=(8,8))
    keys = row['local_path']
    np.random.shuffle(keys)
    for i,a in enumerate(ax.flatten()):
        try:
            a.imshow(plt.imread(keys[i]))
        except:
            pass
        a.axis('off')
    plt.suptitle(f"{row['view_direction']} {row['loss'] :.2f}")

    if save is None:
        plt.show()
    else:
        plt.savefig(save)
        plt.close()

In [61]:
side = set([i[:i.find('Ä')]  for i in os.listdir('tmp/seqs/side/')])
front = set([i[:i.find('Ä')]  for i in os.listdir('tmp/seqs/front/')])
side = df.seq_key.apply(lambda x : x in side)
front = df.seq_key.apply(lambda x : x in front)


In [64]:
(df.loc[front,"view_direction"] == 'Sideways').sum()

40

In [65]:
(df.loc[side,"view_direction"] == 'Forward').sum()

0

In [67]:
df.loc[side,"view_direction"] = 'Sideways'
df.loc[front,"view_direction"] = 'Forward'

In [68]:
len(df)

307

In [72]:
real = pd.read_feather("new_train_seqs.df").set_index('index')
real.loc[df.index] = df
real.reset_index().to_feather("new_train_seqs.df")